In [88]:
import streamlit as st
import numpy as np
import pandas as pd
import tweepy as tp
from datetime import datetime, timedelta
import regex as re

bearerToken = "AAAAAAAAAAAAAAAAAAAAAOUBZwEAAAAAuSI9Lk9VJF5p8oZ60%2Ffnb25FSXo%3DsH2SwTWEqpQOe0acAUZeAiPdazuwZYetImYMSn9Wzk7dmXR1VV"

client = tp.Client(bearer_token=bearerToken, wait_on_rate_limit=True)

def convertDate(tweetObject):
    tweetDate = tweetObject['created_at']
    tweetDate = tweetDate.replace("T", " ")
    tweetDate = str(tweetDate.split(".")[0])
    tweetDate = datetime.strptime(tweetDate, '%Y-%m-%d %H:%M:%S')
    return(tweetDate)



def convertTime(inputDate):

    inputDateSplit = inputDate.split("-")

    year = int(inputDateSplit[0])
    month = int(inputDateSplit[1])
    day = int(inputDateSplit[2])

    if month < 10 and day < 10:
        timeString = f"{year}-0{month}-0{day}T00:00:00Z"
        return timeString
    elif month < 10 and day >= 10:
        timeString = f"{year}-0{month}-{day}T00:00:00Z"
        return timeString
    elif month >= 10 and day < 10:
        timeString = f"{year}-{month}-0{day}T00:00:00Z"
        return timeString
    else:
        timeString = f"{year}-{month}-{day}T00:00:00Z"
        return timeString

def generateTweets(date, df, brand):

    tweets = client.search_recent_tweets(query=brand, tweet_fields=['context_annotations', 'created_at', 'text', 'author_id', 'source', 'public_metrics'], max_results=100, end_time=date)
        
    for tweet in tweets.data:
        tweet_ID = tweet.id
        tweet_content = tweet.text
        tweet_date = tweet.created_at
        tweet_user = tweet.user
        tweet_retweets = tweet.public_metrics['retweet_count']
        tweet_replies = tweet.public_metrics['reply_count']
        tweet_likes = tweet.public_metrics['like_count']

        tweet_quotes = tweet.public_metrics['quote_count']

        df = df.append({'TweetID': str(tweet_ID), 'Content':str(tweet_content), 'Full_Date':tweet_date,
                'Retweets':tweet_retweets, 'Replies':tweet_replies, 'Likes':tweet_likes, 'Quotes':tweet_quotes}, ignore_index=True)

    return df


!pip install streamlit

In [14]:
#Doesn't work: "Love the new", "must try the"
#Potential: favorite snack, favorite drink, love drink, love snack (without quotes)
#Works: "Tried the new", flavor good
phrases = ['flavor good', "Tried the new"]

In [20]:

df = pd.DataFrame(columns=['TweetID', 'Content', 'Full_Date', 'Retweets', 'Replies', 'Likes', 'Quotes'])
for phrase in phrases:
    brand = f'{phrase} -is:retweet'
            

    for i in range(6):
        date = convertTime((datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d"))
        
        df = generateTweets(date, df, brand)


In [87]:
# df.to_excel("phraseAnalyse.xlsx")

df['created_dt'] = df.Full_Date.dt.date
df['created_time'] = df.Full_Date.dt.date


In [86]:
df

,TweetID,Content,Full_Date,Retweets,Replies,Likes,Quotes,Only_Date,Only_Time,tweet_len,stopword_count,hashtag_count,link_count,mention_count,uppercase_count,numeric_count,mention_tags,created_dt
0,1523814395796676609,@owenyaoi WHAT’S ADV.... BUT OMG I HOPE EVERYT...,2022-05-09 23:57:17+00:00,0,1,0,0,2022-05-09,23:57:17,136,1,0,0,1,21,0,[@owenyaoi],2022-05-09
1,1523813828147830784,@PerfumeMountain @HourComeAtLast @ProperDrashi...,2022-05-09 23:55:02+00:00,0,1,2,0,2022-05-09,23:55:02,153,7,0,0,4,0,0,"[@PerfumeMountain, @HourComeAtLast, @ProperDra...",2022-05-09
2,1523812904805445632,"@AmericanEDMGirl Yeah they were good!, birthda...",2022-05-09 23:51:22+00:00,0,0,1,0,2022-05-09,23:51:22,95,6,0,0,1,2,0,[@AmericanEDMGirl],2022-05-09
3,1523812265375457280,@MonkMan2015 Really good my guy. Loads of flav...,2022-05-09 23:48:49+00:00,0,1,2,0,2022-05-09,23:48:49,99,5,0,0,1,0,0,[@MonkMan2015],2022-05-09
4,1523810349820096512,@QuestNutrition Thanks for the reply. Just did...,2022-05-09 23:41:13+00:00,0,0,0,0,2022-05-09,23:41:13,165,11,0,0,1,0,0,[@QuestNutrition],2022-05-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,1521979927649734657,Tried the new Mushroom Jack Burger at @RoysRes...,2022-05-04 22:27:46+00:00,0,1,5,0,2022-05-04,22:27:46,103,3,0,1,1,0,0,[@RoysRestaurants],2022-05-04
1189,1521979651169660931,Tried this new thing at the gym and i must hav...,2022-05-04 22:26:40+00:00,0,1,0,0,2022-05-04,22:26:40,177,16,0,0,0,0,0,[],2022-05-04
1190,1521979176756068352,"Kanye told y’all this is what happened, but y...",2022-05-04 22:24:47+00:00,2,1,2,0,2022-05-04,22:24:47,304,20,0,2,0,0,0,[],2022-05-04
1191,1521979050708901892,@progressive I received an email meant for a c...,2022-05-04 22:24:17+00:00,0,1,0,0,2022-05-04,22:24:17,273,17,0,0,1,2,0,[@progressive],2022-05-04


## Twitter helper functions

In [9]:
#----------------------------------------------
# Load dependencies
#----------------------------------------------
import pandas as pd
import base64
import tweepy as tw
import re
import numpy as np
import string
import unicodedata
import nltk
import gensim
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
import matplotlib.pyplot as plt
import gensim.corpora as corpora
import streamlit as st
from pprint import pprint
from nltk.util import bigrams
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from streamlit_metrics import metric, metric_row
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from warnings import filterwarnings
filterwarnings("ignore")

nltk.download('stopwords')

/Users/vetlefrantzvaag/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vetlefrantzvaag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
#----------------------------------------------
# DEFINE STOPWORDS
#----------------------------------------------

# English stopwords
stopwords_en = nltk.corpus.stopwords.words('english')

In [11]:
# Function 1
# Downloading the .csv from the dashboard
#~~~~~~~~~~~~~~~~~~~~~~~
def get_table_download_link(df):
    # Reference: https://discuss.streamlit.io/t/how-to-download-file-in-streamlit/1806
    """
    Generates a link allowing the data in a given panda dataframe to be downloaded
    in:  dataframe
    out: href string
    """
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # some strings <-> bytes conversions necessary here
    href = f'<a href="data:file/csv;base64,{b64}" download="tweets.csv">Download CSV file</a>'
    return href

In [52]:
dfTest = df.copy()

In [56]:
# Function 2
# Feature extraction (will be useful for further analysis)
#~~~~~~~~~~~~~~~~~~~~~~~
def feature_extraction(df):
    # Extracting the date and time
    df['Only_Date'] = df.Full_Date.dt.date
    df['Only_Time'] = df.Full_Date.dt.time

    #Length of tweet
    df['tweet_len'] = df.Content.apply(lambda x: len(x))

    #Stopword count
    df['stopword_count'] = df.Content.apply(lambda x: len([x for x in x.split() if x in stopwords_en]))

    #Hashtag count
    df['hashtag_count'] = df.Content.apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
    
    #Hashtag count
    df['link_count'] = df.Content.apply(lambda x: len([x for x in x.split() if x.startswith('https')]))

    #Mention count
    df['mention_count'] = df.Content.apply(lambda x: len([x for x in x.split() if x.startswith('@')]))

    #Mention tags
    df['mention_tags'] = df.Content.apply(lambda x: [x for x in x.split() if x.startswith('@')])

    #Numeric count
    df['numeric_count'] = df.Content.apply(lambda x: len([x for x in x.split() if x.isdigit()]))

    #Numeric count
    df['uppercase_count'] = df.Content.apply(lambda x: len([x for x in x.split() if x.isupper()]))

    return df
    

### Finding the most mentioned accounts

In [ ]:

from collections import Counter
countList = []
for mention in df.mention_tags:
    for tag in mention:
        countList.append(tag)
Counter(countList).most_common()

In [83]:
# Function 4 (a)
# Cleaning
#-------------
def round1_text_clean(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) # remove emoji
    text = ' ' + text # added space because there was some weirdness for first word (strip later)
    text = text.lower() # convert all text to lowercase
    text = re.sub(r'(\s)@\w+', '', text) # remove whole word if starts with @
    text = re.sub(r'(\s)\w*\d\w*\w+', '', text) # remove whole word if starts with number
    text = re.sub(r'https\:\/\/t\.co\/*\w*', '', text) # remove https links
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # removes punctuation
    text = re.sub('\[.*?\]', '', text) # removes text in square brackets
    #text = re.sub('\w*\d\w*', '', text) # remove whole word if starts with number
    #text = re.sub(r'(\s)#\w+', '', text) # remove whole word if starts with #
    text = text.strip() # strip text
    return text

In [82]:
# Function 4b
#-------------
text_clean_round1 = lambda x: round1_text_clean(x)

In [ ]:
# Function 5
#-------------
def text_clean_round2(text):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """
    nltk.download('wordnet')
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore'))
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]